<img align="left" src = figs/logos/logo-IJCLab_v1.png width=250, style="padding: 10px"> 
<b>Query information on tracts and patchs from objects table </b> <br>
Last verified to run on 2021-11-18 with LSST Science Pipelines release w_2021_40 <br>
Contact authors: Sylvie Dagoret-Campagne (DP0 Delegate) <br>
Target audience: DP0 delegates member <br>

**Credit:** Originally developed by Sylvie Dagoret-Campagne in the framework provided by Rubin DP0.1 (reference DP0.1 tutorials)

Acknowledgement: Melissa Graham, Leanne Guy, Alex Drlica-Wagner, Keith Bechtol, Grzegorz Madejski, Louise Edwards, and many others ..

### Learning Objectives

The goal is to localize some big structures in tracts by using a Top - Down approach, starting from a tract then selecting manually a redshift slice with Holoview tool.
The density of sources is filtered by a KDE gaussian kernel to emphasize the LSS structure.


### Imports

In [ ]:
# Import general python packages
import numpy as np
import re
import pandas as pd
import pickle
from pandas.testing import assert_frame_equal
import os
import errno
import shutil
import getpass

# Import the Rubin TAP service utilities
from lsst.rsp import get_tap_service, retrieve_query

# LSST Science Pipelines (Stack) packages
import lsst.daf.butler as dafButler
import lsst.afw.display as afwDisplay
import lsst.geom as geom
import lsst.afw.coord as afwCoord
afwDisplay.setDefaultBackend('matplotlib')

#
from lsst import skymap

# Astropy
from astropy import units as u
from astropy.coordinates import SkyCoord

# Bokeh for interactive visualization
import bokeh
from bokeh.io import output_file, output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, HoverTool
from bokeh.plotting import figure
from bokeh.transform import factor_cmap

import holoviews as hv
from holoviews import streams, opts
from holoviews.operation.datashader import rasterize
from holoviews.operation.datashader import datashade, dynspread
from holoviews.plotting.util import process_cmap

import datashader as dsh


# Set the maximum number of rows to display from pandas
pd.set_option('display.max_rows', 20)


# Set the holoviews plotting library to be bokeh
# You will see the holoviews + bokeh icons displayed when the library is loaded successfully
#hv.extension('bokeh')
hv.extension('bokeh', 'matplotlib')

# Display bokeh plots inline in the notebook
output_notebook()

In [ ]:
# What versions of bokeh and holoviews nd datashader are we working with?
# This is important when referring to online documentation as
# APIs can change between versions.
print("Bokeh version: " + bokeh.__version__)
print("Holoviews version: " + hv.__version__)
print("Datashader version: " + dsh.__version__)

In [ ]:
# allow for matplotlib to create inline plots in our notebook
%matplotlib inline
import matplotlib.pyplot as plt      # imports matplotlib.pyplot as plt
from matplotlib.colors import Normalize

import warnings                      # imports the warnings library
import gc                            # imports python's garbage collector

# Ignore warnings
from astropy.units import UnitsWarning
warnings.simplefilter("ignore", category=UnitsWarning)

In [ ]:
# Set up some plotting defaults:

params = {'axes.labelsize': 28,
          'font.size': 24,
          'legend.fontsize': 14,
          'xtick.major.width': 3,
          'xtick.minor.width': 2,
          'xtick.major.size': 12,
          'xtick.minor.size': 6,
          'xtick.direction': 'in',
          'xtick.top': True,
          'lines.linewidth': 3,
          'axes.linewidth': 3,
          'axes.labelweight': 3,
          'axes.titleweight': 3,
          'ytick.major.width': 3,
          'ytick.minor.width': 2,
          'ytick.major.size': 12,
          'ytick.minor.size': 6,
          'ytick.direction': 'in',
          'ytick.right': True,
          'figure.figsize': [18, 10],
          'figure.facecolor': 'White'
          }

plt.rcParams.update(params)

In [ ]:
from astropy.cosmology import FlatLambdaCDM

In [ ]:
from IPython.display import Image

In [ ]:
def remove_figure(fig):
    """Remove a figure to reduce memory footprint. """
    # get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()      # clear the figure
    plt.close(fig) # close the figure
    gc.collect()   # call the garbage collector

In [ ]:
# What version of the Stack are we using?
! echo $IMAGE_DESCRIPTION
! eups list -s | grep lsst_distrib

### 1. Notebook Configuration

#### 1.1 setup pathes

In [ ]:
# username
myusername=getpass.getuser()

In [ ]:
# temporary folders if necessary
NBDIR       = 'tract_lss01'                            # relative path for this notebook output
TMPTOPDIR   = "/scratch"                               # always write some output in /scratch, never in user HOME 
TMPUSERDIR  = os.path.join(TMPTOPDIR,myusername)       # defines the path of user outputs in /scratch 
TMPNBDIR    = os.path.join(TMPUSERDIR,NBDIR)           # output path for this particular notebook

In [ ]:
# create user temporary directory
if not os.path.isdir(TMPUSERDIR):
    try:
        os.mkdir(TMPUSERDIR)
    except:
        raise OSError(f"Can't create destination directory {TMPUSERDIR}!" ) 

In [ ]:
# create this notebook temporary directory
if not os.path.isdir(TMPNBDIR):
    try:
        os.mkdir(TMPNBDIR)
    except:
        raise OSError(f"Can't create destination directory {TMPNBDIR}!" ) 

#### 1.2 Defines steering flags and parameters

The Output queries may be saved in files if requested. 
By defaults all the following flags are set False : no query output is saved in file.
To speed-up the demo, the presenter may keep some of those flags True.


In [ ]:
FLAG_WRITE_DATAFRAMEONDISK  = False                     # Select of query output will be saved on disk
FLAG_READ_DATAFRAMEFROMDISK = False                     # Select of the query can be red from disk if it exists
FLAG_CLEAN_DATAONDISK       = False                     # Select of the output queries saved in file will be cleaned at the end of the notebook

### 2. Explore the DP0.1 Patch structure


The goal is to visualize the tract structure from the object data to check if the data are those described in DC2 paper (as expected).

#### 2.1 Create the Rubin TAP Service client

In [ ]:
# Get an instance of the TAP service
service = get_tap_service()
assert service is not None
assert service.baseurl == "https://data.lsst.cloud/api/tap"

#### 2.2 Get list of tracts

In [ ]:
# Execute the same query using the maxrec parameter instead of the TOP
max_rec=1000
query = "SELECT DISTINCT tract " \
        "FROM dp01_dc2_catalogs.object " \
        "ORDER by tract ASC"
print(query)
filename_result1='tracts_result1.csv'
fullfilename_result1=os.path.join(TMPNBDIR,filename_result1)

In [ ]:
if FLAG_READ_DATAFRAMEFROMDISK and os.path.exists(fullfilename_result1):
    sql_result1 = pd.read_csv(fullfilename_result1,index_col=0)
else:
    sql_result1 = service.search(query, maxrec=max_rec).to_table().to_pandas()
    
if FLAG_WRITE_DATAFRAMEONDISK:
    sql_result1.to_csv(fullfilename_result1)

In [ ]:
sql_result1

In [ ]:
tract_list = sql_result1["tract"].values
tract_list

#### 2.3 Larger query on tracts and counting the number of object in each tract

In [ ]:
query_histogram = "SELECT tract, count(tract) " \
                  " FROM dp01_dc2_catalogs.object " \
                  " GROUP BY tract " \
                  " ORDER BY tract ASC"
print(query_histogram)
filename_result2='tracts_result2.csv'
fullfilename_result2=os.path.join(TMPNBDIR,filename_result2)

In [ ]:
if FLAG_READ_DATAFRAMEFROMDISK and os.path.exists(fullfilename_result2):
    sql_result2 = pd.read_csv(fullfilename_result2,index_col=0)
else:
    sql_result2 = service.search(query_histogram).to_table().to_pandas()
    
if FLAG_WRITE_DATAFRAMEONDISK:
    sql_result2.to_csv(fullfilename_result2)

In [ ]:
sql_histogram = sql_result2

In [ ]:
# add more info to dataframe
sql_histogram['index'] = sql_histogram.index
sql_histogram['logcount'] = np.log10(sql_histogram["count"])

In [ ]:
sql_histogram.head()

In [ ]:
N=len(sql_histogram.index)
print(f"Number of {N} tracts")

#### 2.4 Make a Bar plot with Holoview

##### with tract index along x axis with linear y scale

In [ ]:
%%opts Bars [width=1000, height=200, xrotation= 90, ]

# Custom hover tool for the source detections
tractHoverTool = HoverTool(
    tooltips=[
        ( 'index', '@index{4.0f}'),
        ( 'tract', '@tract{4.0f}'),
        ( 'count', '@count{7.0f}'),
    ],
    formatters={
        'index': 'printf',
        'tract': 'printf',
        'count': 'printf',
    },   
)

hv.Bars(data=sql_histogram, kdims=['index'],vdims=["count","tract","logcount"]).opts(
    title="object distribution in tract (lin scale)",color='darkblue', 
    xlabel='tract number', fontscale=1.2,tools=[tractHoverTool]) 

##### by tract number along x axis with y scale in log

In [ ]:
%%opts Bars [width=1200, height=200, xrotation= 90, ]

# Custom hover tool for the source detections
tractHoverTool = HoverTool(
    tooltips=[
        ( 'tract', '@tract{4.0f}'),
        ( 'count', '@count{7.0f}'),
    ],
    formatters={
        'tract': 'printf',
        'count': 'printf',
    },   
)


hv.Bars(data=sql_histogram, kdims=['tract'],vdims=["logcount","tract","count"]).opts(
    title="object distribution in tract (log scale)",color='darkblue', 
    xlabel='tract number', fontscale=1.2,tools=[tractHoverTool]) 

####  2.5 Compute (RA,DEC) of each patch

In [ ]:
query_multival = "SELECT tract, count(tract), avg(ra) as avg_ra, avg(dec) as avg_dec " \
                  " FROM dp01_dc2_catalogs.object " \
                  " GROUP BY tract " \
                  " ORDER BY tract"
print(query_multival)

filename_result3='tracts_result3.csv'
fullfilename_result3=os.path.join(TMPNBDIR,filename_result3)

In [ ]:
if FLAG_READ_DATAFRAMEFROMDISK and os.path.exists(fullfilename_result3):
    sql_result3 = pd.read_csv(fullfilename_result3,index_col=0)
else:
    sql_result3 = service.search(query_multival).to_table().to_pandas()
    
if FLAG_WRITE_DATAFRAMEONDISK:
    sql_result3.to_csv(fullfilename_result3)

In [ ]:
sql_data3 = sql_result3

In [ ]:
sql_data3['index'] = sql_data3.index
sql_data3['logcount'] = np.log10(sql_data3["count"])

In [ ]:
sql_data3.head()

#### 1.6 2D map of tract objects statistic

##### with Holoview scatter

In [ ]:
# Custom hover tool for the source detections
objHoverTool = HoverTool(
    tooltips=[
        ( 'tract', '@tract{4.0f}'),
        ( 'RA', '@avg_ra{0.2f}'),
        ( 'DEC', '@avg_dec{0.2f}'),
        ( 'count', '@count{7.0f}'),
    ],
    formatters={
        'tract': 'printf',
        'RA' : 'printf',
        'DEC' : 'printf',
        'count' : 'printf',
    },
    
)

In [ ]:
scatter = hv.Scatter(sql_data3, kdims=['avg_ra'],vdims=['avg_dec','tract','count', 'logcount','index'])
scatter = scatter.opts(color='logcount', marker="s",size=10,tools=[objHoverTool])
scatter.hist(dimension=['avg_ra','avg_dec'])

##### with Holoview Points

In [ ]:
zipped_points = np.column_stack((sql_data3['avg_ra'],sql_data3['avg_dec']))

In [ ]:
from holoviews.operation import histogram
points = hv.Points(zipped_points)
radim = hv.Dimension('radim', label='right ascension', unit='deg')
decdim = hv.Dimension('decdim',label='declination', unit='deg')
points.redim(x=radim,y=decdim)
points.opts(color='b', marker="s",size=10,tools=['hover'])
points.hist(dimension=['x','y'])

##### **A very nice piece of code plotting tracts and patches can be found here from Mike**

https://community.lsst.org/t/how-to-make-a-loop-over-tracts-and-patches-to-split-the-dc2-dataset/6081

### 3. Skymap with Butler

- the idea is to search for the geometrical configuration of tracts and patches not based on object positions.
- Low ring tracts aren't uniformly filled by DC2 object, inducing a bias in tract centers.

In [ ]:
Image(filename = "figs/dc2/DC2imprint.png", width=500, height=500)

Sky map of the DC2 footprint. The large green trapezoid is the full DC2 footprint. The
small red square in the upper right corner is the DDF region that is excluded in this release. Each
tract is represented by a rectangle with a number on it showing the tract ID. The light pink region
shows the size of the LSSTCam focal plane as a comparison.

In [ ]:
repo = 's3://butler-us-central1-dp01'
collection = '2.2i/runs/DP0.1'
butler = dafButler.Butler(repo,collections=collection)

In [ ]:
skymap = butler.get('skyMap')

Using Mike Read's snippet
https://community.lsst.org/t/how-to-make-a-loop-over-tracts-and-patches-to-split-the-dc2-dataset/6081/4

In [ ]:
import matplotlib
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

In [ ]:
if 0:
    numTracts=0
    patches = []
    for tractID in tract_list:
        for tractInfo in skymap:
            if tractInfo.getId() == tractID:
               # print(tractID)
                for patch in tractInfo:
                    numTracts+=1
                    #print(patch)
                    ibb=patch.getInnerBBox()
                    tWCS=tractInfo.getWcs()
                    corners = []
                    for corner in ibb.getCorners():
                        p = geom.Point2D(corner.getX(), corner.getY())
                        coord = tWCS.pixelToSky(p)
                        corners.append([coord.getRa().asDegrees(), coord.getDec().asDegrees()])
                    #print(corners)
                    polygon = Polygon(corners,True)
                    patches.append(polygon)
    print(numTracts)
    fig, ax = plt.subplots(figsize=(6,6))
    p = PatchCollection(patches, cmap=matplotlib.cm.jet, alpha=0.4)
    ax.add_collection(p)  
    plt.xlim([47, 77])
    plt.ylim([-47, -25])

### 4. Work at patch level

#### build the query

In [ ]:
def build_query_objintract(tract) -> str:
    
    tract=int(tract)
    query = "SELECT obj.objectId, obj.ra, obj.dec, obj.tract, obj.patch, " \
        "obj.extendedness, obj.good, obj.clean, " \
        "truth.match_objectId, "\
        "truth.redshift, "\
        "truth.truth_type, " \
        "truth.match_sep, truth.is_variable " \
        "FROM dp01_dc2_catalogs.object as obj " \
        "JOIN dp01_dc2_catalogs.truth_match as truth " \
        "ON truth.match_objectId = obj.objectId " \
        "WHERE obj.tract = " + str(tract) + " " \
        "AND truth.match_objectid >= 0 "\
        "AND truth.is_good_match = 1"
    return query
       

#### Select the tract number

In [ ]:
tractID=3264

In [ ]:
query_tract=build_query_objintract(tractID)
print(query_tract)
filename_result4=f'tract{tractID}_result4.pkl'
fullfilename_result4=os.path.join(TMPNBDIR,filename_result4)

In [ ]:
filename_result4

#### Call the job

In [ ]:
if FLAG_READ_DATAFRAMEFROMDISK and os.path.exists(fullfilename_result4):
    sql_result4 = pd.read_pickle(fullfilename_result4)
    tract_data  = sql_result4
else:
    # Create and submit the job. This step does not run the query yet
    job = service.submit_job(query_tract)
    # Get the job URL
    print('Job URL is', job.url)

    # Get the job phase. It will be pending as we have not yet started the job
    print('Job phase is', job.phase)
    
    # Run the job. You will see that the the cell completes executing,
    # even though the query is still running
    job.run()
    
    # Use this to tell python to wait for the job to finish if
    # you don't want to run anything else while waiting
    # The cell will continue executing until the job is finished
    job.wait(phases=['COMPLETED', 'ERROR'])
    print('Job phase is', job.phase)
    
    # A usefull funtion to raise an exception if there was a problem with the query
    job.raise_if_error()
    
    # Once the job completes successfully, you can fetch the results
    async_tract_data = job.fetch_result()
    
    tract_data = async_tract_data.to_table().to_pandas()
    
    
if FLAG_WRITE_DATAFRAMEONDISK:
    tract_data.to_pickle(fullfilename_result4)

In [ ]:
! ls -l $TMPNBDIR

In [ ]:
len(tract_data)

In [ ]:
# map truth_type
tract_data['truth_type']=tract_data['truth_type'].map({1: 'galaxy', 2: 'star', 3: 'SNe'})

In [ ]:
tract_data.head()

#### drop objects that are not galaxies

In [ ]:
tract_data.drop(tract_data.loc[tract_data['truth_type'] != 'galaxy' ].index, inplace=True)

In [ ]:
tract_data.head()

#### 4.1 2D view of the patch (RA,DEC)

In [ ]:
vdims = ['tract','patch','redshift']
ds = hv.Dataset(tract_data, ['ra', 'dec'], vdims)

In [ ]:
ds

##### a) 2D histogram view with matplotlib

In [ ]:
x=tract_data["ra"]
y=tract_data["dec"]
z=tract_data["redshift"]
xmin=x.min()
xmax=x.max()
ymin=y.min()
ymax=y.max()
zmin=z.min()
zmax=z.max()

In [ ]:
H, xedges, yedges = np.histogram2d(x, y, bins=(2000, 2000))

In [ ]:
norm = Normalize(vmin=0, vmax=0.6)

In [ ]:
fig, ax = plt.subplots(figsize=(7,6))
img=ax.imshow(H.T,origin="lower",norm=norm,extent=(xmin,xmax,ymin,ymax),cmap="jet")
plt.colorbar(img, ax=ax)
ax.set_aspect('auto')
ax.set_xlabel("RA (deg)")
ax.set_ylabel("DEC (deg)")
ax.set_title(f"tract {tractID} (matplotlib image, histo2D)")

In [ ]:
remove_figure(fig)

##### b) 2D histogram with hv.Image()

In [ ]:
img_opts = dict(height=350, width=450, 
                xaxis="bottom", 
                padding = 0.01, fontsize={'title': '12pt'},
                colorbar=True, toolbar='right', show_grid=True,
                title= f"tract {tractID} (HV Image, histo2D)",
                xlabel="RA",
                ylabel="DEC",
                tools=['hover']
               )    

In [ ]:
# With hv.Image, must flip up-down the image initialy
flipHT=np.flipud(H.T)
img=hv.Image(flipHT, bounds=(xmin,ymin,xmax,ymax) ).opts(cmap="jet").opts(**img_opts)

In [ ]:
rasterize(img)

##### c) 2D Dynamical map with CDS

###### c.1) Create a CDS

In [ ]:
tract_data.head()

In [ ]:
# Create a column data-source for the plots to share. 
col_data = dict(ra=tract_data["ra"],
                dec=tract_data["dec"],
                z=tract_data["redshift"])
source = ColumnDataSource(data=col_data)

###### c.2)  Dynamical map using CDS (source)

In [ ]:
# Create a holoviews object to hold and plot data
points = hv.Points((source.to_df()['ra'], 
                    source.to_df()['dec'])).opts(
    tools=['box_select', 'lasso_select'])


# Create the linked streams instance
boundsxy = (0, 0, 0, 0)
box = streams.BoundsXY(source=points, bounds=boundsxy)
bounds = hv.DynamicMap(lambda bounds: hv.Bounds(bounds), streams=[box])

# Apply the datashader
p = dynspread(datashade(points,cmap="jet"))
p = p.opts(width=450, height=400,
    padding=0.05, show_grid=True,
    xlim=(xmin, xmax), ylim=(ymin, ymax),
    xlabel="ra", ylabel="dec",title= f"tract {tractID} (HV dynamical map)",)

In [ ]:
# Render the datashaded plot
p * bounds

###### c.3)  Dynamical map using tract_data directly

In [ ]:
# Create a Points Element for the data
points = hv.Points((tract_data["ra"], 
                    tract_data["dec"])).opts(
    tools=['box_select', 'lasso_select'])

# Create the linked streams instance
boundsxy = (0, 0, 0, 0)
box = streams.BoundsXY(source=points, bounds=boundsxy)
bounds = hv.DynamicMap(lambda bounds: hv.Bounds(bounds), streams=[box])

# Apply the datashader
p = dynspread(datashade(points, cmap="Viridis"))
p = p.opts(width=450, height=400,
    padding=0.05, show_grid=True,
    xlim=(xmin, xmax), ylim=(ymin, ymax),
    xlabel="ra", ylabel="dec",title= f"tract {tractID} (HV dynamical map)")

In [ ]:
%%time
# Render the datashaded plot
p * bounds

###### d) CDSView

In [ ]:
# Create a view on truth_type stars
#stars = CDSView(source=source,
#                filters=[GroupFilter(column_name='truth_type', group="star")])
galaxiesradec = CDSView(source=source)

In [ ]:
galaxiesradec

#### 4.2 2D histos with redshift

In [ ]:
img_opts = dict(height=200, width=800, 
                xaxis="bottom", 
                padding = 0.01, fontsize={'title': '12pt'},
                colorbar=True, toolbar='right', show_grid=True,
                tools=['hover']
                #tools=['box_select', 'lasso_select']
                #tools=['hover','box_select']
           )    

In [ ]:
x1=tract_data["redshift"]
y1=tract_data["ra"]
H1, xedges1, yedges1 = np.histogram2d(x1, y1, bins=(1000, 1000))
flipHT1=np.flipud(H1.T)
img1=hv.Image(flipHT1,bounds=(zmin,xmin,zmax,xmax)).opts(cmap="jet",title=f"tract {tractID} : ra vs redshift",xlabel="redshift",ylabel="ra").opts(**img_opts)

In [ ]:
layout1 = rasterize(img1)

In [ ]:
layout1

In [ ]:
x2=tract_data["redshift"]
y2=tract_data["dec"]
H2, xedges2, yedges2 = np.histogram2d(x2, y2, bins=(1000, 1000))
flipHT2=np.flipud(H2.T)
img2=hv.Image(flipHT2,bounds=(zmin,ymin,zmax,ymax)).opts(cmap="jet",title=f"tract {tractID} : dec vs redshift",xlabel="redshift",ylabel="dec").opts(**img_opts)

In [ ]:
layout2 = rasterize(img2)

In [ ]:
layout2

In [ ]:
# FIXME : THIS IS NOT WORKING
#layout = layout1 +layout2 
#layout.cols(1)

#### 4.3 2D Scatter with redshift

In [ ]:
# Select a fraction of input sample
frac = 0.01
tract_data_reduced = tract_data.sample(frac=frac, axis='index')

In [ ]:
len(tract_data_reduced)

In [ ]:
sc1=hv.Scatter(tract_data_reduced,kdims=["redshift"],vdims=["ra"]).opts(
    title=f"tract {tractID} : RA vs redshift distribution",color='darkblue', 
    xlabel='redshift', ylabel='RA',fontscale=1.2,
    height=200, width=800,tools=['hover'])

In [ ]:
sc2=hv.Scatter(tract_data_reduced,kdims=["redshift"],vdims=["dec"]).opts(
    title=f"tract {tractID} : DEC vs redshift distribution",color='darkgreen', 
    xlabel='redshift', ylabel='DEC',fontscale=1.2,tools=['hover'],
    height=200, width=800)

In [ ]:
(sc1+sc2).cols(1)

#### 4.4  Histogram of redshift distribution

In [ ]:
(z_bin, count) = np.histogram(tract_data_reduced['redshift'], bins='fd')
z_distribution = hv.Histogram(z_bin, count).opts(
    title=f"tract {tractID} : redshift distribution",color='darkmagenta', 
    xlabel='redshift', fontscale=1.2,
    height=300, width=600,tools=['hover'])

In [ ]:
z_distribution

#### 4.5 Cosmology

In [ ]:
cosmo = FlatLambdaCDM(H0=70, Om0=0.3, Ob0=None)

In [ ]:
tract_data_cosmo=tract_data

In [ ]:
tract_data_cosmo["dl"]=cosmo.luminosity_distance(tract_data_cosmo["redshift"])

In [ ]:
tract_data_cosmo["da"]=cosmo.angular_diameter_distance(tract_data_cosmo["redshift"])

In [ ]:
tract_data_cosmo["dc"]=cosmo.comoving_distance(tract_data_cosmo["redshift"])
tract_data_cosmo["dra"]= tract_data_cosmo["ra"]-tract_data_cosmo["ra"].mean()
tract_data_cosmo["ddec"]= tract_data_cosmo["dec"]-tract_data_cosmo["dec"].mean()

In [ ]:
tract_data_cosmo["dist_ra"]=tract_data_cosmo["da"]*np.tan(tract_data_cosmo["dra"]*np.pi/180)

In [ ]:
tract_data_cosmo["dist_dec"]=tract_data_cosmo["da"]*np.tan(tract_data_cosmo["ddec"]*np.pi/180)

In [ ]:
img_opts = dict(height=200, width=800, 
                xaxis="bottom", 
                padding = 0.01, fontsize={'title': '12pt'},
                colorbar=True, toolbar='right', show_grid=True,
                tools=['hover']
           )    

In [ ]:
x3=tract_data_cosmo["dc"]
y3=tract_data_cosmo["dist_ra"]
H3, xedges3, yedges3 = np.histogram2d(x3, y3, bins=(1000, 1000))
flipHT3=np.flipud(H3.T)
img3=hv.Image(flipHT3,bounds=(x3.min(),y3.min(),x3.max(),y3.max())).opts(cmap="jet",title=f"tract {tractID} : DT (RA) vs DL",xlabel="longitudinal distance (MPC)",ylabel="transverse distance (MPC)").opts(**img_opts)

In [ ]:
rasterize(img3)

In [ ]:
x4=tract_data_cosmo["dc"]
y4=tract_data_cosmo["dist_dec"]
H4, xedges4, yedges4 = np.histogram2d(x4, y4, bins=(1000, 1000))
flipHT4=np.flipud(H4.T)
img4=hv.Image(flipHT4,bounds=(x4.min(),y4.min(),x4.max(),y4.max())).opts(cmap="jet",title=f"tract {tractID} : DT (DEC) vs DL",xlabel="longitudinal distance (MPC)",ylabel="transverse distance (MPC)").opts(**img_opts)

In [ ]:
rasterize(img4)

In [ ]:
layout = rasterize(img3)+rasterize(img4)

In [ ]:
layout.cols(1)

#### 5. Dynamical map for LSS selection in RA,DEC - redshift plane

In [ ]:
x=tract_data["ra"]
y=tract_data["dec"]
z=tract_data["redshift"]
xmin=x.min()
xmax=x.max()
ymin=y.min()
ymax=y.max()
zmin=z.min()
zmax=z.max()

In [ ]:
# Create a Points Element for the data
points1 = hv.Points((z, 
                    x)).opts(
    tools=['box_select', 'lasso_select'])

# Create the linked streams instance
boundsxy1 = (0, 0, 0, 0)
box1 = streams.BoundsXY(source=points1, bounds=boundsxy1)
bounds1 = hv.DynamicMap(lambda bounds: hv.Bounds(bounds), streams=[box1])

# Apply the datashader
p1 = dynspread(datashade(points1, cmap="viridis"))
p1 = p1.opts(width=800, height=200,
    padding=0.05, show_grid=True,
    xlim=(zmin, zmax), ylim=(xmin, xmax),
    xlabel="redshift", ylabel="RA")

In [ ]:
p1 * bounds1

In [ ]:
box1

In [ ]:
# Create a Points Element for the data
points2 = hv.Points((z, 
                    y)).opts(
    tools=['box_select', 'lasso_select'])

# Create the linked streams instance
boundsxy2 = (0, 0, 0, 0)
box2 = streams.BoundsXY(source=points2, bounds=boundsxy2)
bounds2 = hv.DynamicMap(lambda bounds: hv.Bounds(bounds), streams=[box2])

# Apply the datashader
p2 = dynspread(datashade(points2, cmap="viridis"))
p2 = p2.opts(width=800, height=200,
    padding=0.05, show_grid=True,
    xlim=(zmin, zmax), ylim=(ymin, ymax),
    xlabel="redshift", ylabel="DEC")

In [ ]:
p2 * bounds2

In [ ]:
box2

In [ ]:
# FIXME
#(p1 * bounds1 + p2 * bounds2).cols(1)

##### Select a range of redshifts using box

In [ ]:
#ZMIN_SEL=box1.bounds[0]
#ZMAX_SEL=box1.bounds[2]

#ZMIN_SEL=box2.bounds[0]  
#ZMAX_SEL=box2.bounds[2]  

ZMIN_SEL = 1.74
ZMAX_SEL = 1.847

In [ ]:
ZMIN_SEL,ZMAX_SEL

In [ ]:
ZMEAN_SEL=np.mean([ZMIN_SEL,ZMAX_SEL])
ZMEAN_SEL

In [ ]:
tract_data_SEL = tract_data[ (tract_data["redshift"]> ZMIN_SEL) & (tract_data["redshift"] < ZMAX_SEL) ]

In [ ]:
len(tract_data_SEL)

#### Apply Gaussian kernel (KDE) from stats on selected redshift slice

- idea from https://github.com/rubin-dp0/delegate-contributions-dp01/tree/main/Structure/StructureAroundCluster

In [ ]:
x = tract_data_SEL["ra"]
y = tract_data_SEL["dec"]

In [ ]:
from scipy import stats
xmin = x.min()
xmax = x.max()
ymin = y.min()
ymax = y.max()
#calculate the kde
X, Y = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
positions = np.vstack([X.ravel(), Y.ravel()])
values = np.vstack([x, y])
kernel = stats.gaussian_kde(values)
oZ = np.reshape(kernel(positions).T, X.shape)

#### Plot with matplotlib

In [ ]:
#Make the plot for the RCS structures
f, ax = plt.subplots(1, 1, figsize=(10, 8))
# 
ofils=ax.imshow(np.rot90(oZ), cmap=plt.cm.jet, extent=[xmin, xmax, ymin, ymax])
ax.plot(x, y, 'k.', markersize=2)
f.colorbar(ofils, ax=ax)
#ax.set_xlim(xmin,xmax)
#ax.set_ylim(ymin,ymax)
ax.set_xlabel("RA (deg)")
ax.set_ylabel("DEC (deg)")
ax.set_title(f"LSS at redshift z={ZMEAN_SEL:.2f} in tract {tractID}")
ax.set_aspect("auto")
ax.grid(color="w")

plt.tight_layout()


In [ ]:
remove_figure(f)

#### Plot KDE filter with Holoview

In [ ]:
# Create the Image element.
# Define some default plot options for the Image
img_opts = dict(height=500, width=600, 
                xaxis="bottom", 
                padding = 0.01, fontsize={'title': '12pt'},
                colorbar=True, toolbar='right', show_grid=True,
                tools=['hover']
               )     


bounds_img = (xmin, ymin, xmax, ymax)

img = hv.Image(np.rot90(oZ), bounds=bounds_img,
               kdims=['x', 'y']).opts(
    cmap =plt.cm.jet,  xlabel = 'RA', ylabel ='DEC',
    title = f"LSS at redshift z={ZMEAN_SEL:.2f} in tract {tractID}",
    **img_opts)

##### Holoview with image alone

In [ ]:
rasterize(img)

#### Holoview Image (KDE filter)  + sources

In [ ]:
coords = x,y

In [ ]:
# Custom hover tool for the source detections
detHoverTool = HoverTool(
    tooltips=[
        ( 'RA', '@x{0.2f}'),
        ( 'DEC', '@y{0.2f}'),
    ],
    formatters={
        'RA' : 'printf',
        'DEC' : 'printf',
    },
    
)
objects = hv.Points(coords).opts(
    fill_color=None, size = 1, color="black",
    tools=[detHoverTool])

In [ ]:
# Reset the tools on the image and add a hover on the detections.
rasterize(img).opts(tools=[]) * objects.opts(tools=[detHoverTool])

In the last image, it is possible to zoom on the prominent structure, showing objects and filtered density behind and find approximately the position of one overdensity

In [ ]:
ra_overdens  = 65.9
dec_overdens = -40.44

### 5. Optional : Find the patch and load the corresponding coadd image of a big overdensity

##### 5.1 Fidn the patch corresponding to the overdensity

In [ ]:
def build_query_patchintract(tract) -> str:
    
    tract=int(tract)
    query = "SELECT patch, min(ra) as minRA, max(ra) as maxRA, min(dec) as minDEC , max(dec) as maxDEC, count(*) as count " \
        "FROM dp01_dc2_catalogs.object " \
        "WHERE tract = " + str(tract) + " " \
        "GROUP BY patch " \
        "ORDER BY patch ASC"
    return query

In [ ]:
query_patches = build_query_patchintract(tractID)

In [ ]:
print(query_patches)
filename_result5='tracts_result5.csv'
fullfilename_result5=os.path.join(TMPNBDIR,filename_result5)

In [ ]:
if FLAG_READ_DATAFRAMEFROMDISK and os.path.exists(fullfilename_result5):
    sql_result5 = pd.read_csv(fullfilename_result5,index_col=0)
else:
    sql_result5 = service.search(query_patches).to_table().to_pandas()
    
if FLAG_WRITE_DATAFRAMEONDISK:
    sql_result5.to_csv(fullfilename_result5)

In [ ]:
patch_info = sql_result5
patch_info

In [ ]:
patch_info.describe()

In [ ]:
get_patch=patch_info[((patch_info[ "minRA"]  < ra_overdens ) & ( patch_info[ "maxRA"]  > ra_overdens )) & ((patch_info[ "minDEC"]  < dec_overdens ) & ( patch_info[ "maxDEC"]  > dec_overdens ))]

In [ ]:
get_patch

In [ ]:
patchId=get_patch["patch"].values[0]
patchId

In [ ]:
radec=geom.SpherePoint(ra_overdens, dec_overdens, geom.degrees)

In [ ]:
tractInfo=skymap.findTract(radec)
tractInfo

In [ ]:
patchInfo=tractInfo.findPatch(radec)
patchInfo

#### 5.2 Load the coadd image

Knowing the RA,DEC is sufficient to access to the coadd image (tract and patch id are not necessary)

In [ ]:
# load lsst.afw.display to gain access to image visualization routines.
import lsst.afw.display as afwDisplay
# load the Butler, which provides programmatic access to LSST data products.
from lsst.daf.butler import Butler

In [ ]:
from lsst.afw.image import MultibandExposure

In [ ]:
repo = 's3://butler-us-central1-dp01'
collection = '2.2i/runs/DP0.1'
butler = Butler(repo, collections=collection)

In [ ]:
from astropy.visualization import make_lupton_rgb

In [ ]:
def cutout_coadd(butler, ra, dec, band='r', datasetType='deepCoadd',
                 skymap=None, cutoutSideLength=51, **kwargs):
    """
    Produce a cutout from a coadd at the given ra, dec position.

    Adapted from DC2 tutorial notebook by Michael Wood-Vasey.

    Parameters
    ----------
    butler: lsst.daf.persistence.Butler
        Servant providing access to a data repository
    ra: float
        Right ascension of the center of the cutout, in degrees
    dec: float
        Declination of the center of the cutout, in degrees
    band: string
        Filter of the image to load
    datasetType: string ['deepCoadd']
        Which type of coadd to load.  Doesn't support 'calexp'
    skymap: lsst.afw.skyMap.SkyMap [optional]
        Pass in to avoid the Butler read.  Useful if you have lots of them.
    cutoutSideLength: float [optional]
        Size of the cutout region in pixels.

    Returns
    -------
    MaskedImage
    """
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)

    if skymap is None:
        skymap = butler.get("skyMap")

    # Look up the tract, patch for the RA, Dec
    tractInfo = skymap.findTract(radec)
    patchInfo = tractInfo.findPatch(radec)
    xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    patch = tractInfo.getSequentialPatchIndex(patchInfo)

    coaddId = {'tract': tractInfo.getId(), 'patch': patch, 'band': band}
    parameters = {'bbox': bbox}

    cutout_image = butler.get(datasetType, parameters=parameters,
                              immediate=True, dataId=coaddId)

    return cutout_image

In [ ]:
def createRGB(image, bgr="gri", stretch=1, Q=10, scale=None):
    """
    Create an RGB color composite image.

    Parameters
    ----------
    image : `MultibandExposure`
        `MultibandExposure` to display.
    bgr : sequence
        A 3-element sequence of filter names (i.e., keys of the exps dict)
        indicating what band to use for each channel. If `image` only has
        three filters then this parameter is ignored and the filters
        in the image are used.
    stretch: int
        The linear stretch of the image.
    Q: int
        The Asinh softening parameter.
    scale: list of 3 floats, each less than 1. (default: None)
        Re-scales the RGB channels.

    Returns
    -------
    rgb: ndarray
        RGB (integer, 8-bits per channel) colour image as an NxNx3 numpy array.
    """

    # If the image only has 3 bands, reverse the order of the bands
    #   to produce the RGB image
    if len(image) == 3:
        bgr = image.filters

    # Extract the primary image component of each Exposure with the
    #   .image property, and use .array to get a NumPy array view.

    if scale is None:
        r_im = image[bgr[2]].array  # numpy array for the r channel
        g_im = image[bgr[1]].array  # numpy array for the g channel
        b_im = image[bgr[0]].array  # numpy array for the b channel
    else:
        # manually re-scaling the images here
        r_im = image[bgr[2]].array * scale[0]
        g_im = image[bgr[1]].array * scale[1]
        b_im = image[bgr[0]].array * scale[2]

    rgb = make_lupton_rgb(image_r=r_im,
                          image_g=g_im,
                          image_b=b_im,
                          stretch=stretch, Q=Q)
    # "stretch" and "Q" are parameters to stretch and scale the pixel values

    return rgb

In [ ]:
cutout_image_g = cutout_coadd(butler, ra_overdens , dec_overdens, band='g',
                              datasetType='deepCoadd', cutoutSideLength=601)
cutout_image_r = cutout_coadd(butler, ra_overdens , dec_overdens, band='r',
                              datasetType='deepCoadd', cutoutSideLength=601)
cutout_image_i = cutout_coadd(butler, ra_overdens , dec_overdens, band='i',
                              datasetType='deepCoadd', cutoutSideLength=601)

# Multiband exposures need a list of images and filters
coadds = [cutout_image_g, cutout_image_r, cutout_image_i]
coadds = MultibandExposure.fromExposures(['g', 'r', 'i'], coadds)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10), nrows=1, ncols=2)

title = f"Object in tract {tractID}, patch {patchId}, at redshift {ZMEAN_SEL:.2f}"

# original make_lupton_rgb without any scaling
rgb_original = createRGB(coadds.image, bgr=['g', 'r', 'i'], scale=None)
ax[0].imshow(rgb_original, origin='lower')
ax[0].set_title('original', fontsize=30)

# make_lupton_rgb with scaled rgb channels
ax[1].set_title('re-scaled', fontsize=30)
rgb_scaled = createRGB(coadds.image, bgr=['g', 'r', 'i'],
                       scale=[0.6, 0.7, 1.0])
ax[1].imshow(rgb_scaled, origin='lower')

ax[0].set_axis_off()
ax[1].set_axis_off()

plt.suptitle(title)
plt.show()

# clean up memory
remove_figure(fig)

### 6. Clean the output directory 

In [ ]:
if FLAG_CLEAN_DATAONDISK:
    if os.path.isdir(TMPNBDIR):
        try:
            shutil.rmtree(TMPNBDIR)
        except OSError as e:
            print("Error: %s : %s" % (TMPNBDIR, e.strerror)) 